<a href="https://colab.research.google.com/github/ItWasAllYellow/NLP_2025/blob/main/notebooks/1_word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Word2Vec Implementation from Scratch

This notebook demonstrates how to implement the Word2Vec algorithm from scratch using PyTorch. We'll use the first Harry Potter book as our corpus to train word embeddings.


## 1. Setting Up the Environment

First, we need to import the necessary libraries:
- `torch` and `torch.nn` for tensor operations and neural network functionality
- `string` for string manipulations (removing punctuation)


In [1]:
import torch
import torch.nn as nn
import string


## 2. Getting the Text Data

We'll download the first Harry Potter book to use as our corpus.

In [2]:
!wget "https://raw.githubusercontent.com/amephraim/nlp/master/texts/J.%20K.%20Rowling%20-%20Harry%20Potter%201%20-%20Sorcerer's%20Stone.txt"


--2025-03-20 04:48:17--  https://raw.githubusercontent.com/amephraim/nlp/master/texts/J.%20K.%20Rowling%20-%20Harry%20Potter%201%20-%20Sorcerer's%20Stone.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 439742 (429K) [text/plain]
Saving to: ‘J. K. Rowling - Harry Potter 1 - Sorcerer's Stone.txt’

J. K. Rowling - Har 100%[===================>] 429.44K  --.-KB/s    in 0.05s   

2025-03-20 04:48:17 (9.16 MB/s) - ‘J. K. Rowling - Harry Potter 1 - Sorcerer's Stone.txt’ saved [439742/439742]



## 3. Text Preprocessing

Before we can use the text data, we need to preprocess it:
- Remove punctuation
- Convert text to lowercase
- Split text into tokens (words)

This function will help us clean and tokenize the text.

In [3]:
def remove_punctuation(x):
  return x.translate(''.maketrans('', '', string.punctuation))

def make_tokenized_corpus(corpus):
  out= [ [y.lower() for y in remove_punctuation(sentence).split(' ') if y] for sentence in corpus]
  return [x for x in out if x!=[]]


## 4. Loading and Formatting the Text

Now we'll load the text file, replace some special characters, and split the text into sentences.


In [4]:
with open("J. K. Rowling - Harry Potter 1 - Sorcerer's Stone.txt", 'r') as f:
  strings = f.readlines()
sample_text = "".join(strings).replace('\n', ' ').replace('Mr.', 'mr').replace('Mrs.', 'mrs').split('. ')


Let's tokenize the text using our preprocessing function `make_tokenized_corpus`:

In [5]:
# Corpus is a list of list of strings (words)

In [6]:
corpus = make_tokenized_corpus(sample_text)

corpus[:5]

[['harry',
  'potter',
  'and',
  'the',
  'sorcerers',
  'stone',
  'chapter',
  'one',
  'the',
  'boy',
  'who',
  'lived',
  'mr',
  'and',
  'mrs',
  'dursley',
  'of',
  'number',
  'four',
  'privet',
  'drive',
  'were',
  'proud',
  'to',
  'say',
  'that',
  'they',
  'were',
  'perfectly',
  'normal',
  'thank',
  'you',
  'very',
  'much'],
 ['they',
  'were',
  'the',
  'last',
  'people',
  'youd',
  'expect',
  'to',
  'be',
  'involved',
  'in',
  'anything',
  'strange',
  'or',
  'mysterious',
  'because',
  'they',
  'just',
  'didnt',
  'hold',
  'with',
  'such',
  'nonsense'],
 ['mr',
  'dursley',
  'was',
  'the',
  'director',
  'of',
  'a',
  'firm',
  'called',
  'grunnings',
  'which',
  'made',
  'drills'],
 ['he',
  'was',
  'a',
  'big',
  'beefy',
  'man',
  'with',
  'hardly',
  'any',
  'neck',
  'although',
  'he',
  'did',
  'have',
  'a',
  'very',
  'large',
  'mustache'],
 ['mrs',
  'dursley',
  'was',
  'thin',
  'and',
  'blonde',
  'and',
  'had

## 5. Creating Context Word Pairs

A key concept in Word2Vec is learning from context. We need to create pairs of words that appear near each other in the text. We'll use a sliding window approach to create these pairs.

For example, with the window size of 2, for the word "to" in the sentence "they were the last people youd expect to be involved...", we would create pairs with:
- ("to", "expect")
- ("to", "be")
- ("to", "involved")
- ("to", "in")

These pairs will be our training data.

In [7]:
from tqdm import tqdm

sample_sentence = ['they', 'were', 'the', 'last', 'people', 'youd', 'expect', 'to', 'be', 'involved', 'in', 'anything', 'strange', 'or', 'mysterious', 'because', 'they', 'just', 'didnt', 'hold', 'with', 'such', 'nonsense']

word_pairs = []
window_size = 2

for sample_sentence in tqdm(corpus):
  for cur_idx, center_word in enumerate(sample_sentence):
    window_begin = max(cur_idx - window_size, 0)
    window_end = min(cur_idx + window_size + 1, len(sample_sentence))
    # for context_word in sample_sentence[window_begin:window_end]:
    #   # if center_word == context_word: continue
    #   word_pairs.append( (center_word, context_word))
    for j in range(window_begin, window_end):
      if cur_idx == j: continue
      word_pairs.append( (center_word, sample_sentence[j]))

print(f"\nLength of word_pairs is {len(word_pairs)}")
print(f"First 5 example of word_pairs is {word_pairs[:5]}")

100%|██████████| 4682/4682 [00:00<00:00, 10718.28it/s]


Length of word_pairs is 282372
First 5 example of word_pairs is [('harry', 'potter'), ('harry', 'and'), ('potter', 'harry'), ('potter', 'and'), ('potter', 'the')]


## 6. Building the Vocabulary

To work with word vectors, we need to create a vocabulary that maps each unique word to an index. We'll also filter out rare words that appear less than a certain number of times in the corpus.

### 6.1 Collecting All Words

First, let's collect all words in our corpus:


In [8]:
# we have to make vocabulary
sentence = corpus[0]
entire_words = []

for sentence in corpus:
  for word in sentence:
    entire_words.append(word)




### 6.2 Finding Unique Words

Now, let's find the unique words in our corpus:

In [9]:
# we have to get the "unique" item among total words

unique_words = set(entire_words)
len(unique_words)


6038

### 6.3 Converting to a List and Sorting

We'll convert the set of unique words to a sorted list:

In [10]:
# vocab_set[0] # set is not subscriptable because it has no order

unique_words = sorted(list(unique_words))
unique_words[0]


'\the'

### 6.4 Filtering by Frequency

Now, let's filter out rare words that occur less than a specified number of times:
- We can use the `Counter` class from the `collections` module to count the frequency of each word in the corpus.
- Caution on `alist.sort()` will return `None`.

In [11]:
# how can we filter the vocab by its frequency?
filtered_vocab = None
# you can use word counter as dictionary
# In python dictionary, dict.keys() gives keys, and dict.values() give values,
# dict.items() give (key, value)

from collections import Counter
word_counter = Counter(entire_words)
word_counter.most_common(10)
word_counter['harry']

threshold = 5
filtered_vocab = []

for key, value in word_counter.items():
  if value > threshold:
    filtered_vocab.append(key)

filtered_vocab.sort()
filtered_vocab[:10]

['a',
 'able',
 'abou',
 'about',
 'above',
 'across',
 'added',
 'afford',
 'afraid',
 'after']

## 7. Filtering Word Pairs

Now that we have our filtered vocabulary, we need to filter our word pairs to only include words that are in our vocabulary:

In [12]:
# Filter the word_pairs using the vocab
# word_pairs, filtered_vocab
# word_pairs is a list of [word_a, word_b]

filtered_word_pairs = []
vocab_set = set(filtered_vocab)

for pair in tqdm(word_pairs):
  a, b = pair
  if a in vocab_set and b in vocab_set:
    filtered_word_pairs.append(pair)


100%|██████████| 282372/282372 [00:00<00:00, 591240.48it/s]


In [13]:
# implement same algorithm with list comprehension

filtered_word_pairs = [pair for pair in word_pairs if pair[0] in vocab_set and pair[1] in vocab_set]
filtered_word_pairs[0]

('harry', 'potter')

In [14]:
len(filtered_word_pairs), len(word_pairs)

(226846, 282372)

## 8. Converting Words to Indices

For efficiency, we'll convert our words to indices according to their position in our vocabulary:

In [15]:
# convert word into index of vocab
filtered_vocab.index('harry')

527

This is inefficient because `list.index()` has to scan the list every time. Let's use a dictionary for faster lookups:

In [16]:
# we can make it faster
# use dictionary to find the index of string
word2idx = dict()
for idx, word in enumerate(filtered_vocab):
  word2idx[word] = idx
word2idx['harry']

527

Now, let's convert our word pairs to index pairs more efficiently:

In [17]:
index_pairs = [(word2idx[pair[0]], word2idx[pair[1]]) for pair in filtered_word_pairs]
index_pairs[0]

(527, 953)

In [18]:
# Why we don't need idx2tok?

filtered_vocab[527]

'harry'

## 9. Creating Initial Word Vectors

Now we'll create random vectors for each word in our vocabulary. These vectors will be adjusted during training:
- We can use `torch.randn` to create random vectors that follow normal distribution.

In [19]:
# we have to make random vectors for each word in the vocab
# we also have to decide the dimension of the vector

dim = 100
vocab_size = len(filtered_vocab)

word_vectors = torch.randn(vocab_size, dim) / 10
word_vectors

tensor([[ 0.0715, -0.0867, -0.0370,  ...,  0.0515, -0.0029, -0.1981],
        [-0.0368,  0.0640,  0.1763,  ..., -0.0162,  0.0374,  0.0837],
        [-0.0292,  0.0055,  0.0867,  ...,  0.0789, -0.0685, -0.0400],
        ...,
        [ 0.0012, -0.0674, -0.0859,  ..., -0.0309,  0.0789, -0.1399],
        [-0.1885,  0.0185, -0.0173,  ..., -0.0622,  0.0141, -0.1081],
        [ 0.1098,  0.0485,  0.0431,  ..., -0.0712,  0.0697, -0.0283]])

In [20]:
# what is the vector for harry?

word_vectors[word2idx['harry']]


tensor([ 0.0136, -0.0290, -0.0606,  0.0275,  0.2301, -0.1782, -0.0838, -0.1364,
         0.0145,  0.2419, -0.1684, -0.0582, -0.1133,  0.0925,  0.0273, -0.0387,
         0.0421, -0.0612,  0.0244,  0.0416,  0.0330,  0.1018, -0.0695,  0.0162,
         0.0426,  0.0444, -0.0350, -0.0587, -0.0769, -0.1282, -0.1762,  0.0780,
        -0.0795,  0.0495, -0.0379,  0.1176, -0.0089, -0.0315,  0.1754, -0.1150,
         0.1254, -0.0803,  0.1263,  0.0432, -0.1758,  0.0316, -0.0165, -0.0161,
         0.1275,  0.0675,  0.1224, -0.1058, -0.0645,  0.0711, -0.0343, -0.0758,
        -0.0189, -0.0153,  0.1206, -0.1114, -0.0594, -0.0790,  0.1805, -0.0076,
         0.0230,  0.0514, -0.0165,  0.2476,  0.0861, -0.1447, -0.1245, -0.0581,
         0.0260, -0.0348, -0.0791,  0.1063, -0.0137,  0.0120, -0.0308,  0.1350,
         0.0992,  0.0421,  0.1031,  0.1276, -0.0386, -0.0228, -0.1199, -0.2066,
         0.0185, -0.0188,  0.0179, -0.0224, -0.1020, -0.0632,  0.0479,  0.1080,
        -0.1487,  0.0740, -0.0414, -0.08

## 10. Understanding Word Relationships with Dot Products

The core of Word2Vec is using dot products to measure relationships between words. Let's explore this concept:

In [21]:
torch.set_printoptions(sci_mode=False) # Do this to avoid scientific notation


## Dot Product
- Assume we have two vectors $a$ and $b$.
  - $a = [a_1, a_2, a_3, a_4, ..., a_n]$
  - $b = [b_1, b_2, b_3, b_4, ..., b_n]$
- $a \cdot b$ = $\sum _{i=1}^n a_ib_i$  = $a_1b_1 + a_2b_2 + a_3b_3 + a_4b_4 + ... + a_nb_n$

Let's calculate the dot product between "harry" and "potter":


In [22]:
# calculate P(potter|harry)
harry = word_vectors[word2idx['harry']]
potter = word_vectors[word2idx['potter']]
dot_product_value_between_potter_harry = sum(harry * potter)
dot_product_value_between_potter_harry

tensor(-0.0192)

In [23]:
# we can get the dot product value for every other words in the vocab
# to get  P(word | harry)
word_dot_dict = {}
for word in filtered_vocab:
  w_idx = word2idx[word]
  w_vector = word_vectors[w_idx]
  word_dot_dict[word] = sum(w_vector * harry)
word_dot_dict

{'a': tensor(0.1310),
 'able': tensor(-0.2809),
 'abou': tensor(-0.1048),
 'about': tensor(-0.0037),
 'above': tensor(0.0445),
 'across': tensor(0.0422),
 'added': tensor(-0.1146),
 'afford': tensor(-0.3025),
 'afraid': tensor(0.1291),
 'after': tensor(0.0594),
 'afternoon': tensor(0.0071),
 'again': tensor(0.0221),
 'against': tensor(0.2249),
 'ages': tensor(0.1289),
 'ago': tensor(-0.1055),
 'agreed': tensor(-0.0655),
 'ah': tensor(0.0492),
 'ahead': tensor(-0.0338),
 'air': tensor(0.0683),
 'albus': tensor(0.0829),
 'alive': tensor(0.0049),
 'all': tensor(-0.0074),
 'alley': tensor(-0.0191),
 'allowed': tensor(0.0211),
 'almost': tensor(0.0063),
 'alone': tensor(-0.1591),
 'along': tensor(-0.1859),
 'already': tensor(0.1192),
 'also': tensor(0.0204),
 'although': tensor(-0.1322),
 'always': tensor(-0.2137),
 'am': tensor(-0.2353),
 'an': tensor(-0.0401),
 'and': tensor(0.0664),
 'angrily': tensor(0.1982),
 'angry': tensor(-0.0030),
 'another': tensor(-0.1348),
 'answer': tensor(0.07

Now, let's convert these dot products to probabilities using the softmax function:
- We have to convert our prediction into probability distribution to get P(word|harry) so that sum of [P(a|harry), ..., P(potter|harry), ... P(ron|harry), ... ] = 1
- current dot product value is any real number, sometimes called as logit
  - logit from logistic regression. Some values that are not yet converted to 0-1 or value before sigmoid function
  - every probability should be in range (0, 1) (greater than 0, smaller than 1)
  - this can be handled by taking exponential of dot product values, divided by total sum
  - This function is called **Softmax**

- Why we use exponential?
  - Because we want to make every probability in positive range while preserving the order


In [38]:
from math import exp
word_exp_dict = {}
for word, dot_value in word_dot_dict.items():
  exp_value =torch.exp(dot_value)
  word_exp_dict[word] = exp_value

word_prob_dict = {}
sum_exp = sum([value for value in word_exp_dict.values()]) # 변수명도 convention을 유지하자!
#sum(word_exp_dict.values())
for word, exp_value in word_exp_dict.items():
    word_prob_dict[word] = exp_value / sum_exp

word_prob_dict

{'a': tensor(0.0008),
 'able': tensor(0.0005),
 'abou': tensor(0.0006),
 'about': tensor(0.0007),
 'above': tensor(0.0007),
 'across': tensor(0.0007),
 'added': tensor(0.0006),
 'afford': tensor(0.0005),
 'afraid': tensor(0.0008),
 'after': tensor(0.0007),
 'afternoon': tensor(0.0007),
 'again': tensor(0.0007),
 'against': tensor(0.0008),
 'ages': tensor(0.0007),
 'ago': tensor(0.0006),
 'agreed': tensor(0.0006),
 'ah': tensor(0.0007),
 'ahead': tensor(0.0006),
 'air': tensor(0.0007),
 'albus': tensor(0.0007),
 'alive': tensor(0.0007),
 'all': tensor(0.0007),
 'alley': tensor(0.0006),
 'allowed': tensor(0.0007),
 'almost': tensor(0.0007),
 'alone': tensor(0.0006),
 'along': tensor(0.0005),
 'already': tensor(0.0007),
 'also': tensor(0.0007),
 'although': tensor(0.0006),
 'always': tensor(0.0005),
 'am': tensor(0.0005),
 'an': tensor(0.0006),
 'and': tensor(0.0007),
 'angrily': tensor(0.0008),
 'angry': tensor(0.0007),
 'another': tensor(0.0006),
 'answer': tensor(0.0007),
 'any': tenso

In [25]:
# Get P(potter|harry)
word_prob_dict['potter'] # 이미 harry가 중심어?

## 13. Efficient Matrix Operations
![img](https://mkang32.github.io/images/python/khan_academy_matrix_product.png)

Instead of calculating dot products one by one, we can use matrix multiplication for efficiency:


In [62]:
center_word_mat = torch.stack([harry, potter])
center_word_mat.shape

torch.Size([2, 100])

In [64]:
# get dot product result for every word in the vocabulary
harry.shape

# first, make vector_of_harry into matrix format
harry_mat = harry.unsqueeze(0)
word_vectors.shape

# do matrix multiplication
# dot_by_mat = torch.mm(harry_mat, word_vectors.T) # 뒤집는게 나을 듯
dot_by_mat = torch.mm(center_word_mat, word_vectors.T)
dot_by_mat = dot_by_mat.T
dot_by_mat.shape

torch.Size([1506, 2])

Let's verify that our matrix multiplication gives the same result as individual dot products:

In [65]:
dot_by_mat[word2idx['potter']], word_dot_dict['potter']

(tensor([-0.0192,  1.1364]), tensor(-0.0192))

Now let's implement the complete softmax calculation using matrix operations:


In [66]:
# convert dot product result into exponential
mat_exp = torch.exp(dot_by_mat)
mat_exp.shape

torch.Size([1506, 2])

In [67]:
# get the sum of exponential
sum(mat_exp)

sum_of_mat_exp = torch.sum(mat_exp, dim=0)
sum_of_mat_exp

tensor([1517.0398, 1517.3038])

In [70]:
# divide exponential value with sum
prob = mat_exp / sum(mat_exp) # until now we've set a certain center word, but it has to be chaged
prob.sum(dim=0)

tensor([1.0000, 1.0000])

## 14. Creating a Probability Function

Let's create a function to calculate probabilities efficiently:

In [72]:
def get_probs(query_vectors, entire_vectors):
    dot_by_mat = torch.mm(query_vectors, entire_vectors.T)
    dot_by_mat = dot_by_mat.T

    mat_exp = torch.exp(dot_by_mat)

    sum_of_mat_exp = torch.sum(mat_exp, dim=0)

    prob = mat_exp / sum_of_mat_exp

    return prob

# get_probs(mat_of_harry, word_vectors)
get_probs(center_word_mat, word_vectors)

tensor([[0.0008, 0.0006],
        [0.0005, 0.0008],
        [0.0006, 0.0007],
        ...,
        [0.0007, 0.0006],
        [0.0008, 0.0006],
        [0.0007, 0.0007]])

## 15. Preparing for Training

Before training our Word2Vec model, we need to split our dataset into training and testing sets:

In [78]:
import random

# Now we can train the word2vec

# Let's think about training pairs
index_pairs # this is our dataset. It's list of list of two integer
print(len(index_pairs))
# two integer means a pair of neighboring words

# Training set and Test set
# To validate that our model can solve 'unseen' problems
# So we have to split the dataset before training.

# To randomly split the dataset, we will first shuffle the dataset

random.shuffle(index_pairs) # this will shuffle the list items / it doesen't return the list!

226846


In [79]:
train_set = index_pairs[:200000]
test_set = index_pairs[200000:]

In [80]:
len(train_set), len(test_set)

(200000, 26846)

## 16. Training the Word2Vec Model

Now we'll train our Word2Vec model using batched gradient descent:

In [84]:
# making batch from train_set
# Batch is a set of training samples, that are calculated together
# And also we update the model after one single batch

batch = train_set[:20]
center_words = [x[0] for x in batch]
context_words = [x[1] for x in batch]

# word_vectors[center_words].shape
center_word_vectors = word_vectors[center_words]
prob = get_probs(center_word_vectors, word_vectors)
prob

tensor([[0.0014, 0.0006, 0.0006,  ..., 0.0006, 0.0007, 0.0008],
        [0.0006, 0.0006, 0.0008,  ..., 0.0007, 0.0005, 0.0007],
        [0.0007, 0.0008, 0.0006,  ..., 0.0006, 0.0006, 0.0007],
        ...,
        [0.0007, 0.0006, 0.0006,  ..., 0.0006, 0.0008, 0.0007],
        [0.0007, 0.0007, 0.0007,  ..., 0.0007, 0.0007, 0.0007],
        [0.0007, 0.0006, 0.0006,  ..., 0.0006, 0.0006, 0.0007]])

## 17. Evaluating the Training

Let's visualize the training loss to see if our model is learning:

In [ ]:
import matplotlib.pyplot as plt
plt.plot(loss_record)

## 18. Testing the Model

Now we'll test our model on the test set:

## 19. Exploring Learned Word Relationships

Let's explore what our model has learned by finding the words most closely related to "harry":

In [ ]:
# P(potter|harry)?
